<a href="https://colab.research.google.com/github/way2015cn/way2015cn.github.io/blob/master/%E4%BB%8E%E7%94%A8%E6%88%B7%E5%AE%9E%E9%99%85%E5%BA%94%E7%94%A8%E7%9A%84%E8%A7%92%E5%BA%A6%E5%87%BA%E5%8F%91%EF%BC%8C%E7%94%A8%E6%88%B7%E9%9C%80%E8%A6%81%E5%85%88%E5%AE%9A%E4%B9%89%E4%B8%80%E4%B8%AA%E8%AE%BE%E5%A4%87%EF%BC%8C%E7%BB%99%E5%87%BA%E5%8C%85%E6%8B%AC%E5%A3%81%E5%8E%9A%E5%9C%A8%E5%86%85%E7%9A%84%E5%B0%BA%E5%AF%B8%EF%BC%8C%E8%A3%82%E7%BA%B9%E5%B0%BA%E5%AF%B8%EF%BC%8C%E8%BD%BD%E8%8D%B7%E6%9D%A1%E4%BB%B6%E7%AD%89%EF%BC%8C%E7%84%B6_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

好的，我们按照“说明 + 代码”的模块化结构，并将用户输入置于顶端，以 SI 单位制为默认，重新整理适用于 Jupyter Notebook 的内容。

In [ ]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.14.5
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# # ASME BPVC Section XI Appendix G - 防止失效的断裂韧性准则 (模块化SI版)
#
# **注意:** 本笔记旨在辅助理解 ASME BPVC Section XI Appendix G 的内容和计算方法，**不能替代官方标准文本**。实际工程应用必须严格遵循最新版 ASME Code 的完整要求，并结合合格工程师的专业判断。

# ## 1. 用户输入与设置 (User Inputs & Setup)
#
# **请在此处定义您要分析的设备或工况的参数。默认单位为国际单位制 (SI)。**

import numpy as np
import matplotlib.pyplot as plt
import math

# --- 基本几何与材料参数 ---
# 容器/管道的几何尺寸
vessel_inner_radius_mm = 2000.0  # (Ri) 容器内半径 (mm)
vessel_wall_thickness_mm = 250.0 # (t) 容器壁厚 (mm)

# --- 运行与载荷条件 ---
operating_pressure_mpa = 15.0    # (p) 正常运行压力 (MPa)
analysis_temperature_C = 288.0   # (T) 进行评估的金属温度 (°C)
max_cooldown_rate_C_hr = 50.0    # (CR) 最大冷却速率 (°C/hr), 取正值
max_heatup_rate_C_hr = 50.0      # (HU) 最大升温速率 (°C/hr), 取正值
# 可选：如果需要评估特定弯曲应力
primary_bending_stress_mpa = 10.0 # (σb,primary) 主弯曲应力 (MPa) - 评估G-2214.2时需要

# --- 材料韧性基础参数 ---
RTNDT_u_C = -10.0                # (RTNDT(u)) 未辐照参考零塑性转变温度 (°C)
                                 # 这是评估韧性的基础，需根据材料报告确定

# --- 辐照效应参数 (仅当需要计算 Delta RTNDT 时 relevant for G-2216/G-2500) ---
use_irradiation_effect = True      # 是否考虑辐照影响? (True/False)
fluence_n_cm2 = 2e19             # (Φ) 中子注量 (n/cm^2, E > 1 MeV)
irradiation_temp_C = 290.0       # (Ti) 辐照温度 (°C)
material_Cu_percent = 0.10       # (Cu) 材料铜含量 (wt. %)
material_Ni_percent = 0.60       # (Ni) 材料镍含量 (wt. %)
material_P_percent = 0.012        # (P) 材料磷含量 (wt. %)
# material_type_string: 'forgings'(锻件), 'plates'(板材), 'plates-CE'(CE板材),
#                       'welds'(焊缝), 'Linde 80 welds'(Linde 80焊缝)
material_type_string = 'welds'   # 材料类型

# --- 单位制选择 ---
units_system = 'SI'              # 选择 'SI' 或 'US'

# --- 其他可选参数 ---
# T0_C = -30.0                    # (T0) 材料特定的参考温度 (°C) from ASTM E1921, 用于替代 RTNDT 计算
# hydro_test_temperature_C = 60.0 # 水压试验温度 (°C) - 评估 G-2400/G-2500 时需要

# --- 常量定义 ---
PI = np.pi

print("--- 用户输入 (默认 SI 单位) ---")
print(f"内半径 (Ri): {vessel_inner_radius_mm} mm")
print(f"壁厚 (t): {vessel_wall_thickness_mm} mm")
print(f"运行压力 (p): {operating_pressure_mpa} MPa")
print(f"分析温度 (T): {analysis_temperature_C} °C")
print(f"未辐照 RTNDT (RTNDT(u)): {RTNDT_u_C} °C")
print(f"最大冷却速率 (CR): {max_cooldown_rate_C_hr} °C/hr")
print(f"最大升温速率 (HU): {max_heatup_rate_C_hr} °C/hr")
if use_irradiation_effect:
    print(f"考虑辐照效应: 是")
    print(f"  注量 (Φ): {fluence_n_cm2:.2E} n/cm^2")
    print(f"  辐照温度 (Ti): {irradiation_temp_C} °C")
    print(f"  材料类型: {material_type_string}")
    print(f"  成分: Cu={material_Cu_percent}%, Ni={material_Ni_percent}%, P={material_P_percent}%")
else:
    print(f"考虑辐照效应: 否")
print(f"使用单位制: {units_system}")
print("---------------------------------")

---

### **模块 2: 引言与范围 (Introduction & Scope - G-1100)**

**规范说明 (Normative Explanation):**

ASME Section XI 的非强制性附录 G (Nonmandatory Appendix G) 提出了一个基于线弹性断裂力学 (LEFM) 的分析评估程序。其主要目的是为铁素体承压材料（特别是 RPV）提供防止非延性断裂的准则。程序通过假定保守缺陷，计算 $K_I$，并与材料韧性 $K_{Ic}$ 比较，以确定许用载荷（P-T 曲线）。主要适用于厚度 $\ge$ 100 mm (4 in.) 的铁素体材料。

**Python 代码:**

In [ ]:
# 模块 2: 引言与范围
print("\nASME XI Appendix G - 模块 2: 引言与范围 加载完毕。")
# (无计算代码)

---

### **模块 3: 关键概念与定义 (Key Concepts & Definitions - G-2110, G-2120)**

**规范说明 (Normative Explanation):**

* **$RT_{NDT}$:** 根据 ASME III NB-2331 确定的参考零塑性转变温度，是韧性评估的基准温度。需考虑辐照引起的 $\Delta RT_{NDT}$。
* **$K_{Ic}$:** 平面应变断裂韧性，表示材料抵抗裂纹失稳扩展的能力。附录 G 提供保守的 $K_{Ic}$ 下限曲线。
* **假想缺陷 (G-2120):** 用于保守评估。
    * **标准尺寸 (102mm ≤ t ≤ 305mm):** 深度 $a = t/4$，长度 $\ell = 1.5t$ 的半椭圆表面缺陷。
    * **方向:** 轴向或环向，取决于焊缝或分析位置。
    * **位置:** 内外表面均需考虑。
    * **其他厚度:** $t > 305$mm 用 $t=305$mm 尺寸; $t < 102$mm 用 $a=25.4$mm (1") 深度。

**Python 代码:**

In [ ]:
# 模块 3: 关键概念与定义

# 标准假想缺陷尺寸 (针对 102mm <= t <= 305mm)
POSTULATED_FLAW_DEPTH_FRACTION = 0.25  # a = t/4
POSTULATED_FLAW_LENGTH_FACTOR = 1.5   # l = 1.5 * t
POSTULATED_FLAW_ASPECT_RATIO = (POSTULATED_FLAW_DEPTH_FRACTION /
                                POSTULATED_FLAW_LENGTH_FACTOR) # a/l = 1/6

# 保守缺陷深度 (当 t < 102mm)
POSTULATED_FLAW_DEPTH_CONSERVATIVE_SI = 25.4 # mm

print("\nASME XI Appendix G - 模块 3: 关键概念 加载完毕。")
print(f"标准假想缺陷: a=t/4, l=1.5t (a/l = {POSTULATED_FLAW_ASPECT_RATIO:.4f})")
print(f"t < 102mm 时的保守深度: a = {POSTULATED_FLAW_DEPTH_CONSERVATIVE_SI} mm")

# 计算假想缺陷深度 a (用于后续 K_I 计算)
if units_system == 'SI':
    if vessel_wall_thickness_mm < T_MIN_STANDARD_FLAW_SI:
        postulated_flaw_depth_a_mm = POSTULATED_FLAW_DEPTH_CONSERVATIVE_SI
    elif vessel_wall_thickness_mm > T_MAX_STANDARD_FLAW_SI:
        postulated_flaw_depth_a_mm = T_MAX_STANDARD_FLAW_SI * POSTULATED_FLAW_DEPTH_FRACTION
    else:
        postulated_flaw_depth_a_mm = vessel_wall_thickness_mm * POSTULATED_FLAW_DEPTH_FRACTION
    print(f"用于计算的假想缺陷深度 a = {postulated_flaw_depth_a_mm:.2f} mm")
elif units_system == 'US':
     # Convert thickness to inches for US calculation
    vessel_wall_thickness_in = vessel_wall_thickness_mm / 25.4
    if vessel_wall_thickness_in < T_MIN_STANDARD_FLAW_US:
        postulated_flaw_depth_a_in = POSTULATED_FLAW_DEPTH_CONSERVATIVE_US
    elif vessel_wall_thickness_in > T_MAX_STANDARD_FLAW_US:
        postulated_flaw_depth_a_in = T_MAX_STANDARD_FLAW_US * POSTULATED_FLAW_DEPTH_FRACTION
    else:
        postulated_flaw_depth_a_in = vessel_wall_thickness_in * POSTULATED_FLAW_DEPTH_FRACTION
    print(f"用于计算的假想缺陷深度 a = {postulated_flaw_depth_a_in:.3f} inches")
else:
    postulated_flaw_depth_a_mm = None # Or handle error appropriately
    print("错误：无法确定假想缺陷深度，单位制无效。")

---

### **模块 4: 参考断裂韧性 $K_{Ic}$ (Reference Fracture Toughness $K_{Ic}$ - G-2110, G-2212)**

**规范说明 (Normative Explanation):**

$K_{Ic}$ 参考曲线（图 G-2210-1/1M）给出了材料断裂韧性随温度相对于 $RT_{NDT}$ 变化的下限。

* **计算公式 (G-2110(a)):**
    * SI: $K_{Ic} = 36.5 + 3.0833 \exp[0.036(T - RT_{NDT})]$ (MPa$\sqrt{m}$)
* **适用材料:** 见模块 3 说明。
* **辐照效应:** $\Delta RT_{NDT}$ 需考虑。
* **替代温度 $RT_{T0}$ (G-2110(c)):** $RT_{T0} = T_0 + 19.4$ (°C)。

**Python 代码:**

In [ ]:
# 模块 4: 参考断裂韧性 K_Ic

def calculate_Kic(T_minus_RTNDT, units='SI'):
    """
    计算参考断裂韧性 K_Ic。
    基于 ASME XI Appendix G, G-2110(a) 的解析近似公式。

    参数:
        T_minus_RTNDT (float): 温度相对于 RT_NDT (T - RT_NDT)。单位依赖于 'units' 参数 (°C 或 °F)。
        units (str): 'SI' (MPa*sqrt(m), °C) 或 'US' (ksi*sqrt(in), °F)。默认为 'SI'。

    返回:
        float: 计算得到的 K_Ic 值，若输入无效则为 None。若计算值低于下限则返回下限值。
    """
    if units.upper() == 'SI':
        # SI 单位: K_Ic in MPa*sqrt(m), T in °C
        kic = 36.5 + 3.0833 * np.exp(0.036 * T_minus_RTNDT)
        lower_bound = 29.45 # 近似值，基于图 G-2210-1M
        return max(kic, lower_bound)
    elif units.upper() == 'US':
        # 美制单位: K_Ic in ksi*sqrt(in), T in °F
        kic = 33.2 + 2.806 * np.exp(0.02 * T_minus_RTNDT)
        lower_bound = 26.78 # 近似值，基于图 G-2210-1
        return max(kic, lower_bound)
    else:
        print("错误: 无效的单位制。请使用 'SI' 或 'US'。")
        return None

def calculate_RT_T0(T0, units='SI'):
    """
    基于 T0 计算替代参考温度 RT_T0。
    依据 ASME XI Appendix G, G-2110(c) 公式。

    参数:
        T0 (float): 来自 ASTM E1921 测试的参考温度。单位依赖于 'units' 参数 (°C 或 °F)。
        units (str): 'SI' (°C) 或 'US' (°F)。默认为 'SI'。

    返回:
        float: 计算得到的 RT_T0 值，若输入无效则为 None。
    """
    if units.upper() == 'SI':
        RT_T0 = T0 + 19.4
        return RT_T0
    elif units.upper() == 'US':
        RT_T0 = T0 + 35.0
        return RT_T0
    else:
        print("错误: 无效的单位制。请使用 'SI' 或 'US'。")
        return None

print("\nASME XI Appendix G - 模块 4: 参考断裂韧性 加载完毕。")

# --- 示例计算和绘图 (使用全局输入 RTNDT_u_C) ---
if 'RTNDT_u_C' in locals():
    RTNDT_plot_C = RTNDT_u_C # 未考虑辐照影响的绘图示例
    temps_relative_C = np.linspace(-150, 250, 100) # 相对于RTNDT的温度范围
    temps_abs_C = temps_relative_C + RTNDT_plot_C
    kic_values_si = [calculate_Kic(t_rel, units='SI') for t_rel in temps_relative_C]

    plt.figure(figsize=(7, 5))
    plt.plot(temps_abs_C, kic_values_si, label='$K_{Ic}$ Curve')
    plt.title('参考 $K_{Ic}$ 曲线 (SI 单位)')
    plt.xlabel(f'温度 (°C), ($RT_{{NDT(u)}}$ = {RTNDT_plot_C}°C)')
    plt.ylabel('$K_{Ic}$ (MPa $\sqrt{m}$)')
    plt.grid(True)
    plt.ylim(bottom=0)
    plt.legend()
    plt.show()

    # 示例计算
    T_rel_example = 50 # T - RTNDT = 50 °C
    kic_example = calculate_Kic(T_rel_example, units='SI')
    print(f"示例 (SI): 当 (T - RT_NDT) = {T_rel_example}°C, K_Ic ≈ {kic_example:.2f} MPa*sqrt(m)")
else:
    print("错误: 未在用户输入部分定义 RTNDT_u_C")

---

### **模块 5: $K_I$ 计算 - 膜应力 ($K_{Im}$) (KI Calculation - Membrane Tension - G-2214.1)**

**规范说明 (Normative Explanation):**

压力引起的膜应力 $K_{Im}$ 计算公式为 $K_{Im} = M_m \times (p R_i / t)$。$M_m$ 是几何系数：

* **轴向缺陷 (Axial):** $M_m = 0.04088 \sqrt{t}$ (t in mm, $M_m$ in $\sqrt{m}$)
* **环向缺陷 (Circ):** $M_m = 0.03296 \sqrt{t}$ (t in mm, $M_m$ in $\sqrt{m}$)

**Python 代码:**

In [ ]:
# 模块 5: K_I 计算 - 膜应力 (K_Im)

def calculate_Mm_axial(t, units='SI'):
    """
    计算轴向表面缺陷的 Mm 因子。
    基于 ASME XI Appendix G, G-2214.1 的简化公式 (用于1/4t缺陷)。

    参数:
        t (float): 容器壁厚 (mm 或 inches)。
        units (str): 'SI' (mm) 或 'US' (inches)。默认为 'SI'。

    返回:
        float: Mm 因子 (sqrt(m) 或 sqrt(in))，若输入无效则为 None。
    """
    if t <= 0: return None
    if units.upper() == 'SI':
        Mm = 0.04088 * np.sqrt(t) # sqrt(m)
        return Mm
    elif units.upper() == 'US':
        Mm = 1.29 * np.sqrt(t) # sqrt(in)
        return Mm
    else:
        print("错误: 无效的单位制。请使用 'SI' 或 'US'。")
        return None

def calculate_Mm_circ(t, units='SI'):
    """
    计算环向表面缺陷的 Mm 因子。
    基于 ASME XI Appendix G, G-2214.1 的简化公式 (用于1/4t缺陷)。

    参数:
        t (float): 容器壁厚 (mm 或 inches)。
        units (str): 'SI' (mm) 或 'US' (inches)。默认为 'SI'。

    返回:
        float: Mm 因子 (sqrt(m) 或 sqrt(in))，若输入无效则为 None。
    """
    if t <= 0: return None
    if units.upper() == 'SI':
        Mm = 0.03296 * np.sqrt(t) # sqrt(m)
        return Mm
    elif units.upper() == 'US':
        Mm = 1.04 * np.sqrt(t) # sqrt(in)
        return Mm
    else:
        print("错误: 无效的单位制。请使用 'SI' 或 'US'。")
        return None

def calculate_KIm(p, Ri, t, Mm, units='SI'):
    """
    根据压力、几何形状和 Mm 因子计算 KIm。

    参数:
        p (float): 内压 (MPa 或 ksi)。
        Ri (float): 容器内半径 (mm 或 inches)。
        t (float): 容器壁厚 (mm 或 inches)。
        Mm (float): 来自 calculate_Mm_xxx 的几何因子 (sqrt(m) 或 sqrt(in))。
        units (str): 'SI' 或 'US'。默认为 'SI'。

    返回:
        float: KIm 值 (MPa*sqrt(m) 或 ksi*sqrt(in))，或 None。
    """
    if t <= 0 or Mm is None or Ri <= 0: return None
    if units.upper() == 'SI':
        # p(MPa), Ri(mm), t(mm), Mm(sqrt(m)) -> KIm(MPa*sqrt(m))
        Ri_m = Ri / 1000.0
        t_m = t / 1000.0
        if t_m <= 0: return None
        KIm_val = Mm * (p * Ri_m / t_m)
        return KIm_val
    elif units.upper() == 'US':
        # p(ksi), Ri(in), t(in), Mm(sqrt(in)) -> KIm(ksi*sqrt(in))
        KIm_val = Mm * (p * Ri / t)
        return KIm_val
    else:
        print("错误: 无效的单位制。请使用 'SI' 或 'US'。")
        return None

print("\nASME XI Appendix G - 模块 5: KI Calculation - Membrane Tension (KIm) 加载完毕。")

# Example (using global inputs)
Mm_axial_si = calculate_Mm_axial(vessel_wall_thickness_mm, units_system)
KIm_axial_si = calculate_KIm(operating_pressure_mpa, vessel_inner_radius_mm, vessel_wall_thickness_mm, Mm_axial_si, units_system)

Mm_circ_si = calculate_Mm_circ(vessel_wall_thickness_mm, units_system)
KIm_circ_si = calculate_KIm(operating_pressure_mpa, vessel_inner_radius_mm, vessel_wall_thickness_mm, Mm_circ_si, units_system)

if KIm_axial_si is not None:
    print(f"示例 (SI) 轴向: t={vessel_wall_thickness_mm} mm, p={operating_pressure_mpa} MPa, Ri={vessel_inner_radius_mm} mm -> Mm={Mm_axial_si:.3f} sqrt(m), KIm={KIm_axial_si:.2f} MPa*sqrt(m)")
if KIm_circ_si is not None:
    print(f"示例 (SI) 环向: t={vessel_wall_thickness_mm} mm, p={operating_pressure_mpa} MPa, Ri={vessel_inner_radius_mm} mm -> Mm={Mm_circ_si:.3f} sqrt(m), KIm={KIm_circ_si:.2f} MPa*sqrt(m)")

---

### **模块 6: $K_I$ 计算 - 弯曲应力 ($K_{Ib}$) (KI Calculation - Bending Stress - G-2214.2)**

**规范说明 (Normative Explanation):**

由弯曲应力 $\sigma_{b,max}$ 产生的 $K_{Ib}$ 使用 $K_{Ib} = M_b \times \sigma_{b,max}$ 估算。

* 对于**轴向缺陷**，$M_b \approx (2/3) M_{m, axial}$。
* 远离不连续区时，主弯曲应力通常较小。

**Python 代码:**

In [ ]:
# 模块 6: K_I 计算 - 弯曲应力 (K_Ib)

def calculate_Mb_axial(Mm_axial):
    """ 计算轴向缺陷的 Mb 因子 (基于 Mm_axial)。 """
    if Mm_axial is None: return None
    if Mm_axial < 0: print("警告: Mm_axial 为负，Mb 将为负。")
    return (2.0/3.0) * Mm_axial

def calculate_KIb(sigma_b_max, Mb):
    """ 根据最大弯曲应力和 Mb 因子计算 KIb。 """
    if Mb is None or sigma_b_max is None: return None
    if Mb < 0: print("警告: Mb 为负。检查输入的 Mm。")
    # 假设 sigma_b_max 是导致裂纹张开的拉伸弯曲应力
    # 如果 sigma_b_max 代表压缩，则 KIb 应为 0 或根据约定处理
    return Mb * sigma_b_max

print("\nASME XI Appendix G - 模块 6: KI Calculation - Bending Stress (KIb) 加载完毕。")

# Example for Axial flaw (using previous Mm_axial_si and global bending stress)
Mb_axial_si = calculate_Mb_axial(Mm_axial_si)
KIb_axial_si = calculate_KIb(primary_bending_stress_mpa, Mb_axial_si)

if KIb_axial_si is not None:
    print(f"示例 (SI) 轴向: sigma_b={primary_bending_stress_mpa} MPa, Mm_axial={Mm_axial_si:.3f} -> Mb={Mb_axial_si:.3f} sqrt(m), KIb={KIb_axial_si:.2f} MPa*sqrt(m)")
else:
    print("示例 (SI) 轴向 KIb 计算失败。")

---

### **模块 7: $K_I$ 计算 - 热梯度 ($K_{It}$) (KI Calculation - Thermal Gradient - G-2214.3)**

**规范说明 (Normative Explanation):**

径向热梯度 $K_{It}$ 的简化计算公式基于升/降温速率：

* **内表面 (降温 $C_R$):** $K_{It} = (0.0909 \sqrt{t}) \times C_R$ (t in mm, $M_t$ in MPa$\sqrt{m}$/(°C/hr))
* **外表面 (升温 $HU$):** $K_{It} = (0.1153 \sqrt{t}) \times HU$ (t in mm, $M_t$ in MPa$\sqrt{m}$/(°C/hr))

适用于 $t \ge 102$ mm 且速率 $\le 56$ °C/hr。 G-2214.3(b) 提供更精确方法。

**Python 代码:**

In [ ]:
# 模块 7: K_I 计算 - 热梯度 (K_It)

def calculate_Mt_cooldown(t, units='SI'):
    """ 计算内表面缺陷 (降温) 的 Mt 因子。 """
    if t <= 0: return None
    if units.upper() == 'SI':
        if t < 102.0: print(f"警告: 壁厚 t={t}mm < 102mm，公式 G-2214.3 可能不适用。"); return None
        return 0.0909 * np.sqrt(t) # MPa*sqrt(m)/(°C/hr)
    elif units.upper() == 'US':
        if t < 4.0: print(f"警告: 壁厚 t={t}in < 4in，公式 G-2214.3 可能不适用。"); return None
        return 0.75 * np.sqrt(t) # ksi*sqrt(in)/(°F/hr)
    else:
        print("错误: 无效的单位制。")
        return None

def calculate_Mt_heatup(t, units='SI'):
    """ 计算外表面缺陷 (升温) 的 Mt 因子。 """
    if t <= 0: return None
    if units.upper() == 'SI':
        if t < 102.0: print(f"警告: 壁厚 t={t}mm < 102mm，公式 G-2214.3 可能不适用。"); return None
        return 0.1153 * np.sqrt(t) # MPa*sqrt(m)/(°C/hr)
    elif units.upper() == 'US':
        if t < 4.0: print(f"警告: 壁厚 t={t}in < 4in，公式 G-2214.3 可能不适用。"); return None
        return 0.95 * np.sqrt(t) # ksi*sqrt(in)/(°F/hr)
    else:
        print("错误: 无效的单位制。")
        return None

def calculate_KIt(rate, Mt):
    """ 根据速率 (CR 或 HU) 和 Mt 因子计算 KIt。 """
    if Mt is None or rate is None: return None
    # 速率按正值代入公式
    return Mt * abs(rate)

def calculate_KIt_polynomial(C0, C1, C2, C3, t, is_inside_flaw=True, units='SI'):
    """
    根据多项式应力分布系数计算 KIt (G-2214.3(b))。
    假设 a = t/4。

    参数:
        C0, C1, C2, C3 (float): 多项式应力系数 (MPa, MPa/mm, ... 或 ksi, ksi/in, ...)。
        t (float): 壁厚 (mm 或 inches)。
        is_inside_flaw (bool): 是否为内表面缺陷。
        units (str): 'SI' 或 'US'。默认为 'SI'。

    返回:
        float: KIt 值 (MPa*sqrt(m) 或 ksi*sqrt(in))。
    """
    if t <= 0: return None
    a = t / 4.0 # 这些公式是为 1/4t 缺陷推导的
    pi = np.pi

    if units.upper() == 'SI':
        a_m = a / 1000.0 # 转换为米
        # C1, C2, C3 需要从 MPa/mm^n 转换为 MPa/m^n
        C1_m = C1 * 1000.0
        C2_m2 = C2 * (1000.0**2)
        C3_m3 = C3 * (1000.0**3)
        if is_inside_flaw:
            KIt_val = (1.03*C0 + 0.63*C1_m*a_m + 0.48*C2_m2*a_m**2 + 0.40*C3_m3*a_m**3) * np.sqrt(pi * a_m)
        else: # 外表面缺陷
            KIt_val = (1.04*C0 + 0.63*C1_m*a_m + 0.49*C2_m2*a_m**2 + 0.41*C3_m3*a_m**3) * np.sqrt(pi * a_m)
        return KIt_val # MPa*sqrt(m)
    elif units.upper() == 'US':
        if is_inside_flaw:
            KIt_val = (1.03*C0 + 0.63*C1*a + 0.48*C2*a**2 + 0.40*C3*a**3) * np.sqrt(pi * a)
        else: # 外表面缺陷
            KIt_val = (1.04*C0 + 0.63*C1*a + 0.49*C2*a**2 + 0.41*C3*a**3) * np.sqrt(pi * a)
        return KIt_val # ksi*sqrt(in)
    else:
        print("错误: 无效的单位制。")
        return None

print("\nASME XI Appendix G - 模块 7: KI Calculation - Thermal Gradient (KIt) 加载完毕。")

# Example using simplified rate-based KIt (using global inputs)
Mt_cooldown_si = calculate_Mt_cooldown(vessel_wall_thickness_mm, units_system)
KIt_cooldown_si = calculate_KIt(max_cooldown_rate_C_hr, Mt_cooldown_si)

Mt_heatup_si = calculate_Mt_heatup(vessel_wall_thickness_mm, units_system)
KIt_heatup_si = calculate_KIt(max_heatup_rate_C_hr, Mt_heatup_si)

if KIt_cooldown_si is not None:
    print(f"示例 (SI) 简化冷却 KIt: CR={max_cooldown_rate_C_hr}°C/hr, t={vessel_wall_thickness_mm} mm -> Mt={Mt_cooldown_si:.4f}, KIt={KIt_cooldown_si:.2f} MPa*sqrt(m)")
else:
    print(f"示例 (SI) 简化冷却 KIt 计算失败 (t={vessel_wall_thickness_mm} mm)。")

if KIt_heatup_si is not None:
    print(f"示例 (SI) 简化升温 KIt: HU={max_heatup_rate_C_hr}°C/hr, t={vessel_wall_thickness_mm} mm -> Mt={Mt_heatup_si:.4f}, KIt={KIt_heatup_si:.2f} MPa*sqrt(m)")
else:
    print(f"示例 (SI) 简化升温 KIt 计算失败 (t={vessel_wall_thickness_mm} mm)。")

---

### **模块 8: 许用压力计算 (Allowable Pressure Calculation - G-2215)**

**规范说明 (Normative Explanation):**

G-2215 规定了确定 P-T 运行限制曲线的方法。核心准则是 $2 K_{Im} + K_{It} \le K_{Ic}$。

* **启动:** 由内表面稳态和外表面升温两者中的较小许用压力决定。
* **冷却:** 由内表面降温决定。
* **LTOP:** 与这些限制相关联。

**Python 代码:**

In [ ]:
# 模块 8: 许用压力计算 (G-2215)

def calculate_allowable_pressure(K_Ic, K_It, Mm, Ri, t, units='SI'):
    """
    根据主要准则计算许用压力 P。
    2 * KIm + KIt <= KIc  =>  2 * Mm * (P * Ri / t) + KIt <= KIc
    P <= (KIc - KIt) * t / (2 * Mm * Ri)
    单位必须一致 (SI 或 US)。

    参数:
        K_Ic (float): 目标温度下的参考断裂韧性。
        K_It (float): 热梯度引起的应力强度因子。稳态时可为 0。
        Mm (float): 膜应力的几何因子 (轴向或环向，针对特定表面)。
        Ri (float): 容器内半径 (mm 或 inches)。
        t (float): 容器壁厚 (mm 或 inches)。
        units (str): 'SI' 或 'US'。默认为 'SI'。

    返回:
        float: 许用压力，若计算无法进行则为 None。若 (KIc - KIt) 为负则返回 0。
    """
    if K_Ic is None or Mm is None or Ri <= 0 or t <= 0:
        # print(f"错误: 许用压力计算输入无效 (K_Ic={K_Ic}, Mm={Mm}, Ri={Ri}, t={t})。")
        return None # 返回 None 以便调用者处理
    if K_It is None:
        K_It = 0.0 # 若未提供，假设无热应力

    numerator = K_Ic - K_It

    if units.upper() == 'SI':
        Ri_m = Ri / 1000.0
        t_m = t / 1000.0
        if t_m <= 0 or Mm <= 0: return None
        denominator = 2.0 * Mm * Ri_m / t_m
    elif units.upper() == 'US':
        if t <= 0 or Mm <= 0: return None
        denominator = 2.0 * Mm * Ri / t
    else:
        print("错误: 无效的单位制。")
        return None

    if denominator <= 1e-9: # 避免除以极小或零值
        # print("错误: 许用压力计算中分母过小或为零。")
        return None

    # 物理上许用压力不能为负
    if numerator < 0:
        return 0.0

    allowable_p = numerator / denominator
    return allowable_p

def calculate_operating_limits(Ri, t, RT_NDT, max_CR, max_HU, units='SI'):
    """
    根据 G-2215 计算启动和冷却的 P-T 限制曲线。

    参数:
        Ri (float): 容器内半径。
        t (float): 容器壁厚。
        RT_NDT (float): 参考零塑性转变温度。
        max_CR (float): 最大冷却速率 (绝对值, e.g., 50)。
        max_HU (float): 最大升温速率 (绝对值, e.g., 50)。
        units (str): 'SI' 或 'US'。默认为 'SI'。

    返回:
        tuple: (temperatures, p_allow_startup, p_allow_cooldown)
               temperatures 是温度值的 numpy 数组，
               p_allow_startup, p_allow_cooldown 是对应的许用压力 numpy 数组。
               出错时返回 (None, None, None)。
    """
    # 定义计算的温度范围 (相对于 RT_NDT)
    if units.upper() == 'SI':
        temp_relative = np.linspace(-100, 250, 150) # °C
    elif units.upper() == 'US':
        temp_relative = np.linspace(-150, 450, 150) # °F
    else:
        print("错误: 无效的单位制。")
        return None, None, None

    temps_absolute = temp_relative + RT_NDT # 绝对温度数组
    kic_values = np.array([calculate_Kic(tr, units) for tr in temp_relative])

    # 计算几何和热因子
    Mm_axial_in = calculate_Mm_axial(t, units)
    Mm_circ_in = calculate_Mm_circ(t, units)
    Mm_axial_out = Mm_axial_in # 简化假设
    Mm_circ_out = Mm_circ_in # 简化假设
    Mm_in = max(Mm_axial_in, Mm_circ_in) if Mm_axial_in is not None and Mm_circ_in is not None else None
    Mm_out = max(Mm_axial_out, Mm_circ_out) if Mm_axial_out is not None and Mm_circ_out is not None else None

    Mt_cooldown = calculate_Mt_cooldown(t, units)
    Mt_heatup = calculate_Mt_heatup(t, units)

    if any(v is None for v in [Mm_in, Mm_out, Mt_cooldown, Mt_heatup]):
        print("错误: 计算几何/热因子失败。")
        return None, None, None

    # --- 启动限制 ---
    KIt_heatup_val = calculate_KIt(abs(max_HU), Mt_heatup)
    p_limit_startup_in = np.array([calculate_allowable_pressure(kic, 0, Mm_in, Ri, t, units) for kic in kic_values])
    p_limit_startup_out = np.array([calculate_allowable_pressure(kic, KIt_heatup_val, Mm_out, Ri, t, units) for kic in kic_values])
    # 处理可能的 None 值 (例如 Kic 计算失败或压力计算失败)
    p_limit_startup_in = np.nan_to_num(p_limit_startup_in, nan=np.inf) # Replace None with Inf for min comparison
    p_limit_startup_out = np.nan_to_num(p_limit_startup_out, nan=np.inf)
    p_allow_startup = np.minimum(p_limit_startup_in, p_limit_startup_out)
    p_allow_startup[p_allow_startup == np.inf] = np.nan # Put back NaN if both were None

    # --- 冷却限制 ---
    KIt_cooldown_val = calculate_KIt(abs(max_CR), Mt_cooldown)
    p_allow_cooldown = np.array([calculate_allowable_pressure(kic, KIt_cooldown_val, Mm_in, Ri, t, units) for kic in kic_values])
    p_allow_cooldown = np.nan_to_num(p_allow_cooldown, nan=np.nan) # Keep NaN if calculation failed


    return temps_absolute, p_allow_startup, p_allow_cooldown

print("\nASME XI Appendix G - 模块 8: 许用压力计算 (G-2215) 加载完毕。")

# Example Calculation for Operating Limits (SI Units, using global inputs)
if units_system == 'SI':
    temps_si, p_startup_si, p_cooldown_si = calculate_operating_limits(
        vessel_inner_radius_mm,
        vessel_wall_thickness_mm,
        RTNDT_u_C, # 使用未辐照 RTNDT
        max_cooldown_rate_C_hr,
        max_heatup_rate_C_hr,
        'SI'
    )

    # Plotting the P-T curves (SI Units)
    if temps_si is not None:
        plt.figure(figsize=(8, 6))
        # 过滤掉 NaN 值以便绘图
        valid_startup = ~np.isnan(p_startup_si)
        valid_cooldown = ~np.isnan(p_cooldown_si)
        plt.plot(temps_si[valid_startup], p_startup_si[valid_startup], label=f'启动限制 (HU={abs(max_heatup_rate_C_hr)}°C/hr)')
        plt.plot(temps_si[valid_cooldown], p_cooldown_si[valid_cooldown], label=f'冷却限制 (CR={abs(max_cooldown_rate_C_hr)}°C/hr)')
        plt.xlabel('温度 (°C)')
        plt.ylabel('许用压力 (MPa)')
        plt.title(f'示例 P-T 限制 (SI 单位, $RT_{{NDT(u)}}$={RTNDT_u_C}°C)')
        plt.legend()
        plt.grid(True)
        plt.ylim(bottom=0)
        plt.xlim(left=min(temps_si), right=max(temps_si))
        plt.show()
    else:
        print("为 SI 示例计算 P-T 限制失败。")

---

### **模块 9: 基于风险的许用压力 (Risk-Informed Allowable Pressure - G-2216)**

**规范说明 (Normative Explanation):**

G-2216 提供了一种基于风险的替代 P-T 曲线计算方法。

* **核心公式:** $P_{allow} = \frac{ (\frac{K_{Ic}}{1.1}) - K_{It} }{M_m} \times \frac{t}{R_i}$
* **$\Delta RT_{NDT}$:** 需要根据公式 G-2216(2) （源自 RG 1.99 Rev. 2）计算辐照效应。

**Python 代码:**

In [ ]:
# 模块 9: 基于风险的许用压力 (G-2216)

def calculate_Cue(Cu, Ni, material_type):
    """ 根据 G-2216(2) 计算有效铜含量 Cue。 """
    material_lower = material_type.lower()
    is_linde80_weld = "linde 80" in material_lower and "weld" in material_lower

    if is_linde80_weld and Ni > 0.5:
        Cumax = 0.243
    else:
        Cumax = 0.301
    return min(Cu, Cumax)

def calculate_f(Cue, P):
    """ 根据 G-2216(2) 计算 delta RTNDT 的 f(Cue, P) 项。 """
    term1 = max(0.0, P - 0.008) * 7000.0
    term2_base = max(0.0, Cue - 0.072)
    term2 = min(term2_base, 0.168) * 1510.0 # 使用公式中的显式上限 0.168
    f_val = term1 + term2
    return f_val

def calculate_g(Cue, Ni, Phie):
    """ 根据 G-2216(2) 计算 delta RTNDT 的 g(Cue, Ni, Phie) 项。 """
    if Phie <= 1e-10: # 避免 log10(0) 或极小注量的问题
        return 0.0
    log_phie_ratio = np.log10(Phie / 1e19) # 使用 1e19 n/cm^2 作为参考注量
    # 检查指数计算的有效性
    if (0.272 - 0.0518 * log_phie_ratio) * np.log(Phie / 1e19) > 700: # Prevent overflow
         term_phie = 0 # Set to zero if exponent is too large
    else:
        exponent_phie = 0.272 - 0.0518 * log_phie_ratio
        term_phie = (Phie / 1e19) ** exponent_phie

    term_ni = 0.5 + 0.5 * np.tanh((Ni - 0.812) / 0.538)
    term_cue = 0.5 + 0.5 * np.tanh((Cue - 0.079) / 0.049)
    g_val = term_phie * term_ni * term_cue
    return g_val

def calculate_delta_RTNDT_RG199(fluence, Cu, Ni, P, Ti, material_type, units='SI'):
    """
    根据 G-2216(2) 引用的模型计算 delta RT_NDT。
    注量单位为 n/cm^2 (E > 1 MeV)。 Ti 为辐照温度。

    参数:
        fluence (float): 中子注量 (n/cm^2, E > 1 MeV)。
        Cu (float): 材料铜含量 (wt. %)。
        Ni (float): 材料镍含量 (wt. %)。
        P (float): 材料磷含量 (wt. %)。
        Ti (float): 辐照温度 (°C 或 °F)。
        material_type (str): 'forgings', 'plates', 'plates-CE', 'welds', 'Linde 80 welds'。
        units (str): 'SI' 或 'US'。默认为 'SI'。

    返回:
        float: Delta RT_NDT (°C 或 °F)，若出错则为 None。
    """
    if fluence <= 0: return 0.0 # 无注量无偏移

    material_lower = material_type.lower()
    is_linde80_weld = "linde 80" in material_lower and "weld" in material_lower

    Cue = calculate_Cue(Cu, Ni, material_type)
    f_val = calculate_f(Cue, P)

    if units.upper() == 'SI':
        Ti_C = Ti
        if material_lower == 'forgings': A = 6.333e-8; B = 56.83
        elif material_lower == 'plates-ce': A = 8.672e-8; B = 75.11
        elif material_lower == 'plates': A = 8.672e-8; B = 56.94 # 默认非CE
        elif material_lower == 'welds' or is_linde80_weld: A = 7.872e-8; B = 86.11
        else: print(f"错误: 未知的 material_type '{material_type}'。"); return None

        Phie = fluence * np.exp(-0.00333 * Ti_C)
        if Phie <= 1e-10: return 0.0

        g_val = calculate_g(Cue, Ni, Phie)

        log_phie = np.log10(Phie)
        # CRP 项
        term1_base_si = 32.4 * Ti_C - 198 * log_phie + 2110
        term1_si = A * np.maximum(0, term1_base_si)**1.65 # 确保底数非负
        term2_base_si = 0.28 - 0.10 * log_phie
        term2_exp_si = np.maximum(0, term2_base_si)**0.5 # 确保底数非负
        term3_si = (1 + 56.7 * P) * f_val
        CRP_si = term1_si * term2_exp_si * term3_si
        # CIS 项
        CIS_si = B * g_val
        delta_RTNDT_C = CRP_si + CIS_si
        return delta_RTNDT_C

    elif units.upper() == 'US':
        Ti_F = Ti
        if material_lower == 'forgings': A = 1.140e-7; B = 102.3
        elif material_lower == 'plates-ce': A = 1.561e-7; B = 135.2
        elif material_lower == 'plates': A = 1.561e-7; B = 102.5 # 默认非CE
        elif material_lower == 'welds' or is_linde80_weld: A = 1.417e-7; B = 155.0
        else: print(f"错误: 未知的 material_type '{material_type}'。"); return None

        Phie = fluence * np.exp(-0.00185 * Ti_F)
        if Phie <= 1e-10: return 0.0

        g_val = calculate_g(Cue, Ni, Phie)

        log_phie = np.log10(Phie)
        # CRP 项
        term1_base_us = 18 * Ti_F - 198 * log_phie + 2380
        term1_us = A * np.maximum(0, term1_base_us)**1.65
        term2_base_us = 0.28 - 0.10 * log_phie
        term2_exp_us = np.maximum(0, term2_base_us)**0.5
        term3_us = (1 + 56.7 * P) * f_val
        CRP_us = term1_us * term2_exp_us * term3_us
        # CIS 项
        CIS_us = B * g_val
        delta_RTNDT_F = CRP_us + CIS_us
        return delta_RTNDT_F
    else:
        print("错误: 无效的单位制。")
        return None

def calculate_allowable_pressure_risk_informed(RTNDT_u, delta_RTNDT, T_inlet, max_rate, t, Ri, is_cooldown, material_type_Mm, units='SI'):
    """
    使用 G-2216 方法计算基于风险的许用压力。
    使用 T@1/4t 计算 KIc，使用内表面缺陷的 KIt 公式。

    参数:
        RTNDT_u (float): 未辐照 RTNDT。
        delta_RTNDT (float): 辐照引起的 RTNDT 偏移。
        T_inlet (float): 冷却剂入口温度。
        max_rate (float): 最大冷却(CR)或升温(HU)速率 (绝对值)。
        t (float): 壁厚。
        Ri (float): 内半径。
        is_cooldown (bool): True 表示冷却工况，False 表示升温工况。
        material_type_Mm (str): 用于 Mm 计算的材料类型 ('axial' 或 'circ' 或 'most_limiting')。
        units (str): 'SI' 或 'US'。默认为 'SI'。

    返回:
        float: 许用压力，若出错则为 None。
    """
    if RTNDT_u is None or delta_RTNDT is None: return None
    RTNDT_irr = RTNDT_u + delta_RTNDT

    # *** 简化处理：需要精确的 T@1/4t 计算 ***
    T_at_quarter_t = T_inlet # 使用入口温度作为近似值
    print(f"警告: 使用 T@1/4t ≈ T_inlet ({T_at_quarter_t}) 进行演示。这对于瞬态是不准确的。")
    # ***********************************************

    T_minus_RTNDT_quarter_t = T_at_quarter_t - RTNDT_irr
    K_Ic_quarter_t = calculate_Kic(T_minus_RTNDT_quarter_t, units)
    if K_Ic_quarter_t is None: return None

    # 计算 KIt @ 1/4t (对 HU 和 CD 都使用内表面缺陷/冷却的 Mt 公式，依据 G-2216 的逻辑)
    Mt = calculate_Mt_cooldown(t, units)
    KIt = calculate_KIt(abs(max_rate), Mt) if Mt is not None else 0.0

    # 计算 Mm (使用最保守的，通常是轴向)
    if material_type_Mm.lower() == 'axial':
        Mm = calculate_Mm_axial(t, units)
    elif material_type_Mm.lower() == 'circ':
        Mm = calculate_Mm_circ(t, units)
    elif material_type_Mm.lower() == 'most_limiting':
         Mm_ax = calculate_Mm_axial(t, units)
         Mm_ci = calculate_Mm_circ(t, units)
         if Mm_ax is not None and Mm_ci is not None: Mm = max(Mm_ax, Mm_ci)
         else: Mm = Mm_ax if Mm_ax is not None else Mm_ci
    else:
        print(f"错误: 无效的 material_type_Mm '{material_type_Mm}'。")
        return None

    if Mm is None or Mm <= 0 or Ri <= 0 or t <= 0:
         print(f"错误: 基于风险的压力计算的几何/因子无效 (Mm={Mm}, Ri={Ri}, t={t})。")
         return None

    # 计算基于风险的许用压力 (G-2216 公式)
    # P_allow = ((K_Ic / 1.1) - KIt) * t / (Mm * Ri) (注意单位转换)
    numerator_risk = (K_Ic_quarter_t / 1.1) - KIt
    denominator_risk = 0

    if units.upper() == 'SI':
        Ri_m = Ri / 1000.0
        t_m = t / 1000.0
        if t_m <= 0: return None
        denominator_risk = Mm * Ri_m / t_m
    elif units.upper() == 'US':
        denominator_risk = Mm * Ri / t
    else:
        return None # Should not happen due to earlier checks

    if denominator_risk <= 1e-9: # 避免除零
        print("错误: 基于风险的压力计算中分母过小或为零。")
        return None
    if numerator_risk < 0:
        return 0.0 # 物理上压力不能为负

    allowable_p_risk_factored = numerator_risk / denominator_risk
    return allowable_p_risk_factored

print("\nASME XI Appendix G - 模块 9: 基于风险的许用压力 (G-2216) 加载完毕。")

# Example (using global inputs)
if use_irradiation_effect:
    delta_rtndt_si = calculate_delta_RTNDT_RG199(
        fluence_n_cm2, material_Cu_percent, material_Ni_percent, material_P_percent,
        irradiation_temp_C, material_type_string, 'SI'
    )
else:
    delta_rtndt_si = 0.0 # 不考虑辐照

if delta_rtndt_si is not None:
    print(f"\n计算得到的 Delta RTNDT (SI): {delta_rtndt_si:.1f}°C")
    current_RTNDT_C = RTNDT_u_C + delta_rtndt_si
    print(f"考虑辐照后的 RTNDT = {current_RTNDT_C:.1f} °C")

    # 绘制基于风险的 P-T 曲线
    temps_plot_si_risk = np.linspace(current_RTNDT_C - 75, current_RTNDT_C + 250, 100)
    p_startup_risk_si = np.array([calculate_allowable_pressure_risk_informed(RTNDT_u_C, delta_rtndt_si, T, max_heatup_rate_C_hr, vessel_wall_thickness_mm, vessel_inner_radius_mm, False, 'axial', 'SI') for T in temps_plot_si_risk])
    p_cooldown_risk_si = np.array([calculate_allowable_pressure_risk_informed(RTNDT_u_C, delta_rtndt_si, T, max_cooldown_rate_C_hr, vessel_wall_thickness_mm, vessel_inner_radius_mm, True, 'axial', 'SI') for T in temps_plot_si_risk])

    plt.figure(figsize=(8, 6))
    valid_startup = ~np.isnan(p_startup_risk_si)
    valid_cooldown = ~np.isnan(p_cooldown_risk_si)
    plt.plot(temps_plot_si_risk[valid_startup], p_startup_risk_si[valid_startup], label=f'风险通知启动 (HU={abs(max_heatup_rate_C_hr)}°C/hr)')
    plt.plot(temps_plot_si_risk[valid_cooldown], p_cooldown_risk_si[valid_cooldown], label=f'风险通知冷却 (CR={abs(max_cooldown_rate_C_hr)}°C/hr)')
    plt.xlabel('温度 (°C)')
    plt.ylabel('许用压力 (MPa)')
    plt.title(f'示例风险通知 P-T 限制 (SI, $RT_{{NDT}}$={current_RTNDT_C:.1f}°C)')
    plt.legend()
    plt.grid(True)
    plt.ylim(bottom=0)
    plt.show()

else:
    print("\n无法计算 Delta RTNDT，跳过基于风险的 P-T 曲线绘制。")

---

### **模块 10: 靠近不连续区域的评估 (Evaluation Near Discontinuities - G-2220)**

**规范说明 (Normative Explanation):**

对于接管、法兰等几何不连续区域，应力集中和分布更复杂。

* **应力考虑 (G-2222):** $K_I$ 计算需包含主应力（膜+弯，SF=2）和二次应力（膜+弯，SF=1）及热应力 ($K_{It}$)。
* **接管韧性 (G-2223):** 需在接管内圆角假设特定缺陷，并满足 $2(K_{Ip} + K_{I,ext\_load}) + K_{It} \le K_{Ic}$。计算 $K_I$ 通常需要详细应力分析。

**Python 代码:**

In [ ]:
# 模块 10: 靠近不连续区域的评估 (G-2220)

print("\nASME XI Appendix G - 模块 10: 靠近不连续区域的评估 加载完毕。")
print("此部分的 KI 计算通常需要详细应力分析 (如 FEA)，超出附录 G 简化公式范围。")
# (无通用计算代码)

---

### **模块 11: 水压试验温度 (Hydrostatic Test Temperature - G-2400, G-2500)**

**规范说明 (Normative Explanation):**

为防止水压试验时发生非延性断裂，规定了最低试验温度。

* **燃料加载前 (G-2400(a)):** $T_{test} \ge RT_{NDT} + 33°C$ (60°F)。
* **燃料加载后 (G-2400(b)):** 需 LEFM 分析，确保 $1.5 (K_{Im,p} + K_{Ib,p}) + (K_{Im,s} + K_{Ib,s}) \le K_{Ic}$。
* **基于风险的替代方法 (G-2500):** 在升/降温速率 $\le 22°C/hr$ (40°F/hr) 时，可使用类似 G-2216 的方法计算 P-T 限制，但压力项的安全系数为 1.0 (通过 $K_{Ic} = K_{Im} + K_{It}$ 计算)。

**Python 代码:**

In [ ]:
# 模块 11: 水压试验温度 (G-2400, G-2500)

def check_hydro_temp_pre_fuel(T_test, RT_NDT, units='SI'):
    """ 检查水压试验温度是否满足燃料加载前的建议值。 """
    if units.upper() == 'SI':
        T_limit = RT_NDT + 33.0
        print(f"燃料加载前建议值: T_test >= {RT_NDT}°C + 33°C = {T_limit}°C")
        return T_test >= T_limit
    elif units.upper() == 'US':
        T_limit = RT_NDT + 60.0
        print(f"燃料加载前建议值: T_test >= {RT_NDT}°F + 60°F = {T_limit}°F")
        return T_test >= T_limit
    else:
        print("错误: 无效的单位制。")
        return False

def check_hydro_KI_post_fuel(KI_applied_hydro, T_test, RT_NDT, units='SI'):
    """ 检查施加的 KI 是否满足燃料加载后水压试验的 KIc 限制。 """
    # KI_applied_hydro = 1.5*KIm_p + 1.5*KIb_p + KIm_s + KIb_s
    # 注意: 此函数假设 KI_applied_hydro 已经通过详细分析计算得出。
    T_minus_RTNDT = T_test - RT_NDT
    KIc_at_Ttest = calculate_Kic(T_minus_RTNDT, units)
    if KIc_at_Ttest is None: return False
    print(f"燃料加载后检查: T_test={T_test}, T-RT_NDT={T_minus_RTNDT:.1f}, KIc={KIc_at_Ttest:.2f}")
    print(f"施加的 KI (水压, 主应力 SF=1.5) = {KI_applied_hydro:.2f}")
    return KI_applied_hydro <= KIc_at_Ttest

def calculate_hydro_test_pressure_risk_informed(RTNDT_u, delta_RTNDT, T_inlet, max_rate, t, Ri, is_cooldown, material_type_Mm, units='SI'):
    """
    使用 G-2520 方法计算基于风险的水压试验许用压力。
    公式为 KIc = KIm + KIt (压力项隐含 SF=1.0)。

    参数:
        (同 calculate_allowable_pressure_risk_informed)

    返回:
        float: 水压试验条件的许用压力，若出错则为 None。
    """
    if RTNDT_u is None or delta_RTNDT is None: return None
    RTNDT_irr = RTNDT_u + delta_RTNDT

    # *** 简化处理: 需要精确的 T@1/4t 计算 ***
    T_at_quarter_t = T_inlet
    print(f"警告: 使用 T@1/4t ≈ T_inlet ({T_at_quarter_t}) 进行演示。这对于瞬态是不准确的。")
    # ***********************************************

    T_minus_RTNDT_quarter_t = T_at_quarter_t - RTNDT_irr
    K_Ic_quarter_t = calculate_Kic(T_minus_RTNDT_quarter_t, units)
    if K_Ic_quarter_t is None: return None

    # 计算 KIt @ 1/4t (使用内表面缺陷/冷却的 Mt 公式)
    Mt = calculate_Mt_cooldown(t, units)
    KIt = calculate_KIt(abs(max_rate), Mt) if Mt is not None else 0.0

    # 计算 Mm (使用最保守的)
    if material_type_Mm.lower() == 'axial':
        Mm = calculate_Mm_axial(t, units)
    elif material_type_Mm.lower() == 'circ':
        Mm = calculate_Mm_circ(t, units)
    elif material_type_Mm.lower() == 'most_limiting':
         Mm_ax = calculate_Mm_axial(t, units); Mm_ci = calculate_Mm_circ(t, units)
         if Mm_ax is not None and Mm_ci is not None: Mm = max(Mm_ax, Mm_ci)
         else: Mm = Mm_ax if Mm_ax is not None else Mm_ci
    else:
        print(f"错误: 无效的 material_type_Mm '{material_type_Mm}'。")
        return None

    if Mm is None or Mm <= 0 or Ri <= 0 or t <= 0:
         print(f"错误: 基于风险的水压压力计算的几何/因子无效。")
         return None

    # 计算水压试验的许用压力 (SF=1.0)
    # P = (K_Ic - KIt) * t / (Mm * Ri)
    numerator_risk_hydro = K_Ic_quarter_t - KIt
    denominator_risk_hydro = 0

    if units.upper() == 'SI':
        Ri_m = Ri / 1000.0
        t_m = t / 1000.0
        if t_m <= 0: return None
        denominator_risk_hydro = Mm * Ri_m / t_m
    elif units.upper() == 'US':
        denominator_risk_hydro = Mm * Ri / t
    else:
        return None

    if denominator_risk_hydro <= 1e-9: return None
    if numerator_risk_hydro < 0: return 0.0

    allowable_p_hydro_risk_sf1 = numerator_risk_hydro / denominator_risk_hydro
    return allowable_p_hydro_risk_sf1

print("\nASME XI Appendix G - 模块 11: 水压试验温度 加载完毕。")

# Example Checks (using global inputs, SI)
if units_system == 'SI':
    print(f"\n检查燃料加载前水压试验温度: T_test={hydro_test_temperature_C}°C, RT_NDT(u)={RTNDT_u_C}°C")
    is_ok_pre_fuel_si = check_hydro_temp_pre_fuel(hydro_test_temperature_C, RTNDT_u_C, 'SI')
    print(f" 是否满足建议值? {is_ok_pre_fuel_si}")

    # 燃料加载后检查需要预先计算的 KI_applied_hydro
    KI_applied_hydro_example_si = 60.0 # MPa*sqrt(m) (假设值)
    T_test_post_fuel_C = 100 # °C
    current_RTNDT_C = RTNDT_u_C + (delta_rtndt_si if 'delta_rtndt_si' in locals() and delta_rtndt_si is not None else 0)
    print(f"\n检查燃料加载后水压试验温度/KI: T_test={T_test_post_fuel_C}°C, 当前 RT_NDT={current_RTNDT_C:.1f}°C")
    is_ok_post_fuel_si = check_hydro_KI_post_fuel(KI_applied_hydro_example_si, T_test_post_fuel_C, current_RTNDT_C, 'SI')
    print(f" 是否满足准则? {is_ok_post_fuel_si}")

    # 示例：基于风险的水压试验压力
    if 'delta_rtndt_si' in locals() and delta_rtndt_si is not None:
         p_hydro_risk_cooldown_si = calculate_hydro_test_pressure_risk_informed(
            RTNDT_u_C, delta_rtndt_si, hydro_test_temperature_C, max_cooldown_rate_C_hr,
            vessel_wall_thickness_mm, vessel_inner_radius_mm, True, 'axial', 'SI'
        )
         if p_hydro_risk_cooldown_si is not None:
            print(f"示例：基于风险的水压试验许用压力 (冷却, T_test={hydro_test_temperature_C}°C): {p_hydro_risk_cooldown_si:.3f} MPa")
    else:
        print("\n未计算 Delta RTNDT，跳过基于风险的水压试验压力示例。")

---

### **模块 12: 结论与免责声明 (Conclusion & Disclaimer)**

**规范说明 (Normative Explanation):**

ASME Section XI, 非强制性附录 G 提供了一套重要的 LEFM 方法学，用于评估铁素体压力容器的结构完整性，防止非延性断裂。它通过 P-T 曲线和水压试验要求，确保在各种运行工况下的安全裕度。

**重要声明:**

* 此笔记仅为教学和理解目的，**绝不能替代 ASME BPVC Section XI 官方标准**。
* 实际工程应用必须基于最新、最完整的官方规范，并由合格工程师执行。
* 本笔记中的 Python 实现主要基于附录 G 中的简化公式。
* 所有计算的准确性和适用性取决于输入的准确性和对 Code 的正确理解。
* **务必参考最新版的 ASME BPVC Section XI 标准进行任何实际工程评估。**

**Python 代码:**

In [ ]:
# 模块 12: 结论与免责声明

print("\nASME XI Appendix G - 模块 12: 结论与免责声明 加载完毕。")
print("本 Jupyter 笔记提供了对附录 G 的模块化解读和计算实现。")
print("*** 免责声明：请务必参考官方 ASME 规范进行实际工程决策。***")